In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../../msds434-whoop-app-44384939c1f4.json")

project_id = 'msds434-whoop-app'
client = bigquery.Client(credentials=credentials, project=project_id)

In [4]:
# daily data
query_job = client.query(
    """ 
    SELECT

r.cycle_id
, r.sleep_id
, date(r.created_at) recovery_date
, extract(week from r.created_at at TIME ZONE 'America/Chicago') as week_of_year
, extract(dayofweek from r.created_at at TIME ZONE 'America/Chicago') as day_of_week
, r.score_state
, r.score_recovery_score as recovery_score
, r.score_resting_heart_rate as resting_heart_rate
, r.score_hrv_rmssd_milli as hrv_milli
, r.score_spo2_percentage as spo2_perc
, r.score_skin_temp_celsius as skin_temp_celsius
, s.score_state
, s.start as sleep_start
, s.end as sleep_end
, case when extract(hour from s.start AT TIME ZONE 'America/Chicago') < 12
 then extract(hour from s.start AT TIME ZONE 'America/Chicago') + 24
 else extract(hour from s.start AT TIME ZONE 'America/Chicago' ) end +
 extract(minute from s.start AT TIME ZONE 'America/Chicago')/60.0 as sleep_start_time
, s.score_stage_summary_total_in_bed_time_milli as in_bed_time
, s.score_stage_summary_total_no_data_time_milli as no_data_time
, s.score_stage_summary_total_light_sleep_time_milli as light_sleep_time
, s.score_stage_summary_total_slow_wave_sleep_time_milli as slow_wave_sleep_time
, s.score_stage_summary_total_rem_sleep_time_milli as rem_sleep_time
, s.score_stage_summary_sleep_cycle_count as sleep_cycle_count
, s.score_stage_summary_disturbance_count as disturbance_count
, s.score_sleep_needed_baseline_milli as sleep_need
, s.score_sleep_needed_need_from_sleep_debt_milli as sleep_need_from_debt
, s.score_sleep_needed_need_from_recent_strain_milli as sleep_need_from_strain
, s.score_respiratory_rate as respiratory_rate
, s.score_sleep_performance_percentage as sleep_performance_perc
, s.score_sleep_consistency_percentage as sleep_consistency_perc
, s.score_sleep_efficiency_percentage as sleep_efficiency_perc
, s.score sleep_score

, c.score_average_heart_rate as y_avg_heart_rate
, c.score_max_heart_rate as y_max_heart_rate
, c.score_kilojoule as y_kilojoule
, c.score_strain as y_strain

, sy.score_state as y_score_state
, sy.score_stage_summary_total_in_bed_time_milli as y_in_bed_time
, sy.score_stage_summary_total_no_data_time_milli as y_no_data_time
, sy.score_stage_summary_total_light_sleep_time_milli as y_light_sleep_time
, sy.score_stage_summary_total_slow_wave_sleep_time_milli as y_slow_wave_sleep_time
, sy.score_stage_summary_total_rem_sleep_time_milli as y_rem_sleep_time
, sy.score_stage_summary_sleep_cycle_count as y_sleep_cycle_count
, sy.score_stage_summary_disturbance_count as y_disturbance_count
, sy.score_sleep_needed_baseline_milli as y_sleep_need
, sy.score_sleep_needed_need_from_sleep_debt_milli as y_sleep_need_from_debt
, sy.score_sleep_needed_need_from_recent_strain_milli as y_sleep_need_from_strain
, sy.score_respiratory_rate as y_respiratory_rate
, sy.score_sleep_performance_percentage as y_sleep_performance_perc
, sy.score_sleep_consistency_percentage as y_sleep_consistency_perc
, sy.score_sleep_efficiency_percentage as y_sleep_efficiency_perc
, sy.score y_sleep_score

, cy.score_average_heart_rate as y_avg_heart_rate
, cy.score_max_heart_rate as y_max_heart_rate
, cy.score_kilojoule as y_kilojoule
, cy.score_strain as y_strain

 FROM `msds434-whoop-app.whoopdataset.recovery` r
 left join `msds434-whoop-app.whoopdataset.sleep` s on s.id = r.sleep_id
  left join `msds434-whoop-app.whoopdataset.cycles` c on c.id = r.cycle_id
  left join `msds434-whoop-app.whoopdataset.sleep` sy on date(sy.created_at, "America/Chicago") = date(r.created_at, "America/Chicago") - 1
  left join `msds434-whoop-app.whoopdataset.cycles` cy on date(cy.created_at, "America/Chicago") = date(r.created_at, "America/Chicago") - 1
where s.nap = FALSE
and sy.nap = FALSE
    """
)

whoop_daily = query_job.to_dataframe()

In [5]:
# add workouts

In [ ]:
# calculate weekly avgs, acute-to-chronic ratios

In [6]:
whoop_daily

,cycle_id,sleep_id,recovery_date,week_of_year,day_of_week,score_state,recovery_score,resting_heart_rate,hrv_milli,spo2_perc,...,y_sleep_need_from_strain,y_respiratory_rate,y_sleep_performance_perc,y_sleep_consistency_perc,y_sleep_efficiency_perc,y_sleep_score,y_avg_heart_rate_1,y_max_heart_rate_1,y_kilojoule_1,y_strain_1
0,295732012,546671256,2022-11-12,45,7,SCORED,1.0,72.0,30.418001,95.833336,...,302265.0,16.230469,92.0,65.0,88.213090,None,67,129,8384.3670,6.747776
1,270316801,504894343,2022-09-06,36,3,SCORED,33.0,63.0,55.321415,97.777780,...,517984.0,17.812500,74.0,37.0,92.081520,None,78,182,8764.5610,12.381544
2,240368447,456582765,2022-06-18,24,7,SCORED,1.0,63.0,35.580547,98.500000,...,587684.0,17.167969,37.0,58.0,91.854190,None,88,156,14460.6020,16.654894
3,221623469,426415802,2022-04-30,17,7,SCORED,90.0,44.0,104.515370,97.578950,...,2544430.0,17.695312,68.0,72.0,97.732100,None,62,173,9066.1190,10.653727
4,263336612,493626026,2022-08-18,33,5,SCORED,52.0,46.0,84.505226,94.125000,...,1352684.0,15.937500,65.0,78.0,92.772500,None,64,171,9116.3530,11.704348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,182839640,364333223,2022-01-12,2,4,SCORED,58.0,50.0,74.734740,96.375000,...,344836.0,17.109375,72.0,81.0,96.699850,None,66,86,1993.7882,0.000000
1198,182839640,364333223,2022-01-12,2,4,SCORED,58.0,50.0,74.734740,96.375000,...,344836.0,17.109375,72.0,81.0,96.699850,None,79,185,2949.0120,10.707949
1199,308041960,566292459,2022-12-15,50,5,SCORED,70.0,52.0,74.451454,97.833336,...,286965.0,16.933594,91.0,81.0,90.272780,None,67,165,9345.8920,10.863384
1200,299614407,552820783,2022-11-22,47,3,SCORED,45.0,54.0,65.214660,99.000000,...,2503527.0,17.109375,73.0,79.0,94.549034,None,71,176,9867.6130,12.384622
